In [1]:
from datetime import datetime
from concurrent import futures
import pandas as pd
from pandas import DataFrame
from pathlib import Path
import os
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

In [2]:
nasdaq = [
'AMD',
'ADBE',
'ABNB',
'ALGN',
'AMZN',
'AMGN',
'AEP',
'ADI',
'ANSS',
'AAPL',
'AMAT',
'ASML',
'TEAM',
'ADSK',
'ATVI',
'ADP',
'AZN',
'AVGO',
'BIDU',
'BIIB',
'BMRN',
'BKNG',
'CDNS',
'CHTR',
'CPRT',
'CRWD',
'CTAS',
'CSCO',
'CMCSA',
'COST',
'CSX',
'CTSH',
'DDOG',
'DOCU',
'DXCM',
'DLTR',
'EA',
'EBAY',
'ENPH',
'EXC',
'FAST',
'META',
'FISV',
'FTNT',
'GILD',
'GOOG',
'GOOGL',
'HON',
'ILMN',
'INTC',
'INTU',
'ISRG',
'MRVL',
'IDXX',
'JD',
'KDP',
'KLAC',
'KHC',
'LRCX',
'LCID',
'LULU',
'MELI',
'MAR',
'MTCH',
'MCHP',
'MDLZ',
'MRNA',
'MNST',
'MSFT',
'MU',
'NFLX',
'NTES',
'NVDA',
'NXPI',
'ODFL',
'ORLY',
'PCAR',
'PANW',
'PAYX',
'PDD',
'PYPL',
'PEP',
'QCOM',
'REGN',
'ROST',
'SIRI',
'SGEN',
'SPLK',
'SWKS',
'SBUX',
'SNPS',
'TSLA',
'TXN',
'TMUS',
'VRSN',
'VRSK',
'VRTX',
'WBA',
'WDAY',
'XEL',
'ZM',
'ZS'
]

bad_names = []

In [3]:
len(nasdaq)

102

In [4]:
dirpath = os.path.join(Path().resolve(), 'NASDAQ_100')
Path(dirpath).mkdir(parents=True, exist_ok=True)

In [5]:
def get_stock_data(stock):
    
    """ try to query the iex for a stock, if failed note with print """
    try:
        print(stock)
        stock_df = pdr.get_data_yahoo(stock, start_time, now_time)
        stock_df.index = pd.to_datetime(stock_df.index, format='%Y-%m-%d')
        stock_df.index = stock_df.index.tz_localize(None)
        output_name = os.path.join(dirpath, stock) + '.csv'
        stock_df.to_csv(output_name)
    except:
        bad_names.append(stock)
        print('bad: %s' % (stock))
        

In [8]:
# Schrödinger's cat

def get_stock_data_optimal(stock):
    
    """ try to query the iex for a stock, if failed note with print """
    try:
        
        print(stock)
        stock_df = pd.read_csv(os.path.join(Path().resolve(), 'NASDAQ_100', stock) + '.csv', index_col='Date')
        last_date = datetime.strptime(stock_df.tail(1).index.to_list()[0], '%Y-%m-%d')
        now_time = datetime.now().date()
        start_time = datetime(last_date.year, last_date.month , last_date.day + 1)
        
        if pdr.get_data_yahoo(stock, start_time, now_time).shape[0] != 0:
            start_time = datetime(last_date.year, last_date.month , last_date.day + 1)
            stock_diff = pdr.get_data_yahoo(stock, start_time, now_time)
            stock_all = pd.concat([stock_df, stock_diff], ignore_index=False, axis=0)            
            cutten_days = abs((now_time - start_time).days) + 1
            stock_all.iloc[cutten_days]
            output_name = os.path.join(dirpath, stock) + '.csv'
            stock_all.to_csv(output_name)
        
    except:
        
        bad_names.append(stock)
        print('bad: %s' % (stock))

In [11]:
datetime.now().date()

datetime.date(2023, 1, 18)

In [7]:
if __name__ == '__main__':

    """ set the download window """
    now_time = datetime.now()
    start_time = datetime(now_time.year - 1, now_time.month , now_time.day)

    """here we use the concurrent.futures module's ThreadPoolExecutor
        to speed up the downloads buy doing them in parallel 
        as opposed to sequentially """

    #set the maximum thread number
#     max_workers = 50

#     workers = min(max_workers, len(s_and_p)) #in case a smaller number of stocks than threads was passed in
#     with futures.ThreadPoolExecutor(workers) as executor:
#         res = executor.map(get_stock_data, s_and_p)

    for i in nasdaq:
        get_stock_data(i)

    
    """ Save failed queries to a text file to retry """
    if len(bad_names) > 0:
        with open(os.path.join(dirpath, 'failed_queries.txt'),'w') as outfile:
            for name in bad_names:
                outfile.write(name+'\n')

    #timing:
    finish_time = datetime.now()
    duration = finish_time - now_time
    minutes, seconds = divmod(duration.seconds, 60)
    print('bad_names: ', end='')
    print(bad_names)
    print(f'Script took {minutes} minutes and {seconds} seconds to run.')


AMD
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ADBE
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ABNB
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ALGN
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
AMGN
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
AEP
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
A

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
LRCX
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
LCID
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
LULU
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
MELI
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
MAR
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
MTCH
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
MCHP
